In [2]:
import sys
sys.path.append('../')

from typing import List, Dict, Tuple, Optional, Union
import os
import requests
from glob import glob
import json
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from dataset.utils import *

In [3]:
DATA_DIR = "/opt/datasets/thebackend"
BASE_URL = "http://ec2-13-125-225-205.ap-northeast-2.compute.amazonaws.com"
PUSH_URL = BASE_URL + "/push-data"
LOGIN_URL = BASE_URL + "/login-data"

MONTHS_STR = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

PUSH_DATA = os.path.join(DATA_DIR, "push.parquet")

In [3]:
push_df = download_push(save_dir=DATA_DIR, overwrite=False, verbose=True)
print("Number of pushes:", len(push_df))
push_df.head()

File /opt/datasets/thebackend/push.parquet.gzip already exists. Skipping...
Number of pushes: 41523


,timestamp,game_id,is_ad
0,2019-09-24 12:00:00,2147484420,False
1,2019-09-24 12:30:00,2147484420,False
2,2019-09-24 15:30:00,2147484420,False
3,2019-10-11 23:00:00,748,False
4,2019-10-26 12:00:00,438,False


In [4]:
push_df['game_id'].unique()

array([2147484420,        748,        438,         72,        822,
       2147484504,        521,         85,        385,        892,
              232,        527,        427,        237,        807,
              975,        410,        525,        432,       1009,
             1007,        931,        398,        833, 2147484583,
             1191,        283,        824,        796,        797,
             1068,       1391,       1135,       1484,       1510,
             1440,       1341,       1337,        105,        120,
             1515,       1758,       1744,       1783,       1413,
             1591,       1104,       1482,        941,       1779,
             1494,       1368,       1940, 2147484610,       1951,
             1693,       2062,       1865,       2123,       2097,
             1760,       2080,       2253,       1638,       1933,
             2304,       2267,       2195,       1903,       2280,
             1853,       2364,       1088,       1802,       2

In [5]:
start_date = datetime(2022, 4, 1)
end_date = datetime(2022, 8, 1)

push_df = push_df[(push_df['timestamp'] >= start_date) & (push_df['timestamp'] < end_date)]
print(len(push_df))

11753


In [6]:
BEFORE_HOUR = 12
AFTER_HOUR = 6
SAMPLING_DAY = 7

avail_push = get_available_pushes(push_df, before_hour=BEFORE_HOUR, after_hour=AFTER_HOUR, sampling_day=SAMPLING_DAY)
# need data period prior to sampling day for T=0
avail_push = avail_push[(avail_push['timestamp'] >= start_date + timedelta(days=SAMPLING_DAY)) & (avail_push['is_ad'] == True)]
print("Number of available pushes:", len(avail_push))
avail_push.head()

Number of available pushes: 54


,timestamp,game_id,is_ad
21961,2022-04-16 12:00:00,2097,True
22052,2022-04-17 12:00:00,2097,True
22322,2022-04-20 11:44:52,2097,True
23263,2022-04-30 12:00:00,2097,True
23361,2022-05-01 12:00:00,2097,True


In [7]:
avail_push['game_id'].unique()

array([2097, 3065, 1585])

In [8]:
login_csvs_04 = sorted(glob(os.path.join(DATA_DIR, '04', 'raw', 'login', '*.csv')))
login_csvs_07 = sorted(glob(os.path.join(DATA_DIR, '07', 'raw', 'login', '*.csv')))

login_04 = preprocess_login_csv(login_csvs_04)
login_07 = preprocess_login_csv(login_csvs_07)

print("Number of login records in APR:", len(login_04))
print("Number of login records in JUL:", len(login_07))

Number of login records in APR: 9409802
Number of login records in JUL: 6208411


In [9]:
login_04.head()

,timestamp,game_id,gamer_id
0,2022-04-01 09:00:01,2708,E3F2684504FE4207EE3368B6229A4ECC3793C56524AE12...
1,2022-04-01 09:00:01,1916,E0EA56AE51C70ABF1611742BECFE98B09EC1BC7D00CA6C...
2,2022-04-01 09:00:01,2097,6E3D6673357B36BB53949B29CC6FC4AEB4777E0168A51B...
3,2022-04-01 09:00:02,380,837F2759F1521D3615D10AB402CE6152A13DA9BCCB8499...
4,2022-04-01 09:00:02,2273,694EA7080F9FB31FC631DFE4275A8A5D97D17C4F7677E6...


In [10]:
login_07.head()

,timestamp,game_id,gamer_id
0,2022-07-01 09:00:00,1908,B7657AA237DA01E842FB89C8FE2FB772947E4BD88D3DCE...
1,2022-07-01 09:00:00,2326,77FC00DAE87D6FEF670F8352D7FBEF5B06ADBEBD8B8F10...
2,2022-07-01 09:00:00,380,9945989258E03007D67C6C08872FD6F013A55885CE0CFD...
3,2022-07-01 09:00:01,2734,883D5407B7ED923BCEAFD01946690CFA42F7F640EF0B3C...
4,2022-07-01 09:00:02,2082,3437349B74170BA0F537FBAC6BCC75FE436EAC65B4CDBF...


In [11]:
login_05 = load_login_from_parquet(os.path.join(DATA_DIR, '05', 'login.parquet'))
login_06 = load_login_from_parquet(os.path.join(DATA_DIR, '06', 'login.parquet'))

print("Number of login records in MAY:", len(login_05))
print("Number of login records in JUN:", len(login_06))

Number of login records in MAY: 7682737
Number of login records in JUN: 6347094


In [12]:
login_05.head()

,timestamp,game_id,gamer_id
0,2022-05-01 00:00:00.281,2870,D15813658C080ED71B604F9652497E80E584EBB0663041...
1,2022-05-01 00:00:00.838,2870,25B0FE66B0AAF9C658C97BD296F59E738DF80AA67C13CE...
2,2022-05-01 00:00:01.137,1999,DCA1CEE21AA84B5D53722BFF4C56DCD6EA62B1B7269F6D...
3,2022-05-01 00:00:01.416,2273,A15946CC33F014162DEB35B9FCAFCF2AF4F3F779128E3A...
4,2022-05-01 00:00:01.450,2273,E916EBA787250EAB46F0C78786CD6A60C74B8554B9B852...


In [13]:
login_06.head()

,timestamp,game_id,gamer_id
0,2022-06-01 00:00:00.303,2273,EF4FAD5942C1660ED9DA8F35011F856D98D3A73CE944F8...
1,2022-06-01 00:00:00.364,2082,3E3BE17527ACDFED07B66D525763526C9BB65C9517970C...
2,2022-06-01 00:00:00.634,2326,6156AD175A9878A69A89C4DBCC72A30FB5FD71AFF78C25...
3,2022-06-01 00:00:01.139,428,403C1450B8C6C294BD35A7F9591B4F612ACDF40BC14323...
4,2022-06-01 00:00:01.257,2147,4A2E276E7CCA32EE34E20F7044035BCB3F73B6DFD97B95...


In [19]:
login_df = pd.concat([login_04, login_05, login_06, login_07], axis=0)
login_df.reset_index(drop=True, inplace=True)
login_df['gamer_id'] = login_df['gamer_id'].astype(str)
login_df['game_id'] = login_df['game_id'].astype('category')

In [20]:
login_df.head()

,timestamp,game_id,gamer_id
0,2022-04-01 09:00:01,2708,E3F2684504FE4207EE3368B6229A4ECC3793C56524AE12...
1,2022-04-01 09:00:01,1916,E0EA56AE51C70ABF1611742BECFE98B09EC1BC7D00CA6C...
2,2022-04-01 09:00:01,2097,6E3D6673357B36BB53949B29CC6FC4AEB4777E0168A51B...
3,2022-04-01 09:00:02,380,837F2759F1521D3615D10AB402CE6152A13DA9BCCB8499...
4,2022-04-01 09:00:02,2273,694EA7080F9FB31FC631DFE4275A8A5D97D17C4F7677E6...


In [25]:
login_df.to_parquet(os.path.join(DATA_DIR, 'login_APR_JUL.parquet.gzip'), index=False, compression='gzip', engine='pyarrow')

In [10]:
login_df = pd.read_parquet(os.path.join(DATA_DIR, 'login_APR_JUL.parquet.gzip'), engine='pyarrow')

In [11]:
login_df.head()

,timestamp,game_id,gamer_id
0,2022-04-01 09:00:01,2708,E3F2684504FE4207EE3368B6229A4ECC3793C56524AE12...
1,2022-04-01 09:00:01,1916,E0EA56AE51C70ABF1611742BECFE98B09EC1BC7D00CA6C...
2,2022-04-01 09:00:01,2097,6E3D6673357B36BB53949B29CC6FC4AEB4777E0168A51B...
3,2022-04-01 09:00:02,380,837F2759F1521D3615D10AB402CE6152A13DA9BCCB8499...
4,2022-04-01 09:00:02,2273,694EA7080F9FB31FC631DFE4275A8A5D97D17C4F7677E6...


In [28]:
login_df.dtypes

timestamp    datetime64[ns]
game_id               int64
gamer_id             object
dtype: object

In [4]:
crud_dfs = []
for month in range(4, 8):
    crud_df = pd.read_parquet(os.path.join(DATA_DIR, f'{month:02d}', 'raw', f'{month:02d}.parquet'), engine='pyarrow')
    crud_df.rename(columns={'indate': 'timestamp', 'inDate': 'timestamp'}, inplace=True)
    crud_df['timestamp'] = crud_df['timestamp'].dt.tz_localize(None)
    crud_df.sort_values(by=['timestamp'], inplace=True)
    crud_dfs.append(crud_df)
crud_df = pd.concat(crud_dfs, axis=0)

In [12]:
crud_df['timestamp'].min(), crud_df['timestamp'].max()

(Timestamp('2022-04-01 00:00:00.030000'),
 Timestamp('2022-07-06 23:59:59.822000'))

In [13]:
crud_df['game_id'].unique()

array([1585, 2097, 2195, 1829, 3065, 3322, 3548])

In [7]:
crud_df.reset_index(drop=True, inplace=True)

In [9]:
crud_df.to_parquet(os.path.join(DATA_DIR, 'crud_APR_JUL.parquet.gzip'), index=False, compression='gzip', engine='pyarrow')

In [16]:
crud_df.memory_usage(deep=True).sum() / 1024**3

24.534302121028304

In [17]:
login_df.memory_usage(deep=True).sum() / 1024**3

3.782829415053129